In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config")
os.environ.setdefault("DJANGO_CONFIGURATION", "Local")

import configurations
configurations.setup()

# import django
from django.conf import settings
# settings.configure(DEBUG=True)
# django.setup()


In [2]:
# Now this script or any imported module can use any part of Django it needs.
from djstruct.models import DjangoBaseNode, DjangoDependencyRelation

In [3]:
import yaml
IMPORD_DATA_DIR = '/Users/ivan/Projects/Minireference/STRUCTURE/structure-api/data/miniref'
context_yaml_file = os.path.join(IMPORD_DATA_DIR,'context.yml')
context = yaml.load(open(context_yaml_file))

data_filenames = ['math.yml', 'physics.yml']
all_data = []
for filename in data_filenames:
    this_yaml_file = os.path.join(IMPORD_DATA_DIR, filename)
    this_data = yaml.load(open(this_yaml_file))
    all_data += this_data


In [4]:
len(all_data)

22

In [7]:
# YAML PARSING HELPER CLASSES
################################################################################

class PrerequisteReference(object):
    """
    A wrapper that accepts either a string or a dict prerequisite data.
    """
    def __init__(self, data):
        if isinstance(data, basestring):
            self.prerequisite = data
            self.explain_prerequisite = None
            self.level = None
        elif isinstance(data, dict):
            self.prerequisite = data.get('prerequisite')
            self.explain_prerequisite = data.get('explain', None)
            self.level = data.get('level', None)
        else:
            raise ValueError('data must be string or dict')


class UsedforReference(object):
    """
    A wrapper that accepts either a string or a dict usedfor data.
    """
    def __init__(self, data):
        if isinstance(data, basestring):
            self.usedfor = data
            self.explain_usedfor = None
            self.level = None
        elif isinstance(data, dict):
            self.usedfor = data.get('usedfor')
            self.explain_usedfor = data.get('explain', None)
            self.level = data.get('level', None)
        else:
            raise ValueError('data must be string or dict')


class RelatedReference(object):
    """
    A wrapper that accepts either a string or a dict relation data.
    """
    def __init__(self, source, data):
        if source is None:
            raise ValueError('must give relation source path')
        self.source = source
        if isinstance(data, basestring):
            self.related = data
            self.explain = None
            self.level = None
        elif isinstance(data, dict):
            self.related = data.get('related')
            self.explain = data.get('explain', None)
            self.level = data.get('level', None)
        else:
            raise ValueError('data must be string or dict')


            
# MAIN NODE YAML PARSING CLASS
################################################################################

class NodeFromYamlDict(object):
    """
    A class to handle loading a Node from it's YAML serarializaiton.
    Usage:
        node = NodeFromYamlDict(dict)
        node.path / node.description / etc. are now available (or None)
    """
    def __init__(self, data):
        if not data['path']:
            raise ValueError('Yaml data must have a `path`.')
        self.path = data['path']
        # aliases
        self.aliases = []
        if 'aliases' in datum and datum['aliases'] is not None:
            for alias in datum['aliases']:
                self.aliases.append(alias)
        #
        # optional properties: level, description, books
        optional_props = ['level', 'description', 'books']
        for prop_name in optional_props:
            if prop_name in data and data[prop_name] is not None:
                setattr(self, prop_name, data[prop_name])   
        #
        # process prereqs and usedfors
        self.prerequisites = []
        self.usedfors = []
        if 'prerequisites' in datum and datum['prerequisites'] is not None:
            for prereq_str_or_dict in datum['prerequisites']:
                prereq = PrerequisteReference(prereq_str_or_dict)
                self.prerequisites.append(prereq)
        if 'usedfors' in datum and datum['usedfors'] is not None:
            for usedfor_str_or_dict in datum['usedfors']:
                usedfor = UsedforReference(usedfor_str_or_dict)
                self.usedfors.append(usedfor)
        #
        # relations
        self.relations = []
        if 'relations' in datum and datum['relations'] is not None:
            for relation_str_or_dict in datum['usedfors']:
                relation = RelatedReference(self.path, relation_str_or_dict)
                self.relations.append(relation)
        #
        # TODO: __class__, ispartof/contents 
        # also ccmms-specific ones: ccss_guid, ccss_url, asn_url

In [8]:
# inflate data prerequisites into objects
data_objects = []

# also, get a list of all paths and all the paths with refs
all_paths = set()
all_path_refs = set()

for datum in all_data:
    obj = NodeFromYamlDict(datum)
    all_paths.add(obj.path)
    
    for prereq in obj.prerequisites:
        all_path_refs.add(prereq.prerequisite)
    for ufor in obj.usedfors:
        all_path_refs.add(ufor.usedfor)
    for rel in obj.relations:
        all_path_refs.add(rel.related)
    
    data_objects.append(obj)

In [10]:
# PASS 0. delete all nodes
for oldnode in DjangoBaseNode.objects.all():
    oldnode.delete()
for oldreln in DjangoDependencyRelation.objects.all():
    oldreln.delete()

    
# PASS 1. create nodes
for obj in data_objects:
    node = DjangoBaseNode(
        path=obj.path,
        scope='miniref'
    )
    node.save()

# PASS 2. create relations
for obj in data_objects:
    node = DjangoBaseNode.objects.get(path=obj.path)
    for prereq in obj.prerequisites:
        prereq_path = prereq.prerequisite
        if prereq.prerequisite in all_paths:
            prereq_node = DjangoBaseNode.objects.get(path=prereq_path)
            # both ends exist...
            rel = DjangoDependencyRelation(
                    prerequisite=prereq_node,
                    usedfor=node,
                    explain_prerequisite=prereq.explain_prerequisite
            )
            if prereq.level is not None:
                rel.level = prereq.level
            rel.save()

In [11]:
print( len( DjangoBaseNode.objects.all() ) )

print( len( DjangoDependencyRelation.objects.all() ) )
# all_paths

22
32


In [ ]:
data_objects[2].__dict__

In [13]:
#### UNRELATED PROLOG STUFF

perl_funciton_name = 'prerequisites_for'

def slashes_to_underscores(subj):
    return subj.replace('/','_')

def knowify(subj):
    return 'K_'+slashes_to_underscores(subj)

def learnify(subj):
    return 'L_'+slashes_to_underscores(subj)

def make_prolog_statement(subject):
    prereqs_list = []
    if 'prerequisites' in subject and subject['prerequisites'] is not None:
        for prereq_str_or_dict in subject['prerequisites']:
            prereq = PrerequisteReference(prereq_str_or_dict)
            prereqs_list.append(prereq.prerequisite)
    prolog_statement = ''
    prolog_statement += perl_funciton_name + '('
    prolog_statement += '['
    prolog_statement += ','.join( map(knowify, prereqs_list) )
    prolog_statement += '],'
    prolog_statement += learnify(subject['path'])
    prolog_statement += ').'
    return prolog_statement

for subject in all_data:
    print make_prolog_statement(subject)


prerequisites_for([K_math_numbers,K_math_functions,K_math_algebra],L_math_calculus).
prerequisites_for([K_math_functions,K_math_algebra],L_math_calculus_derivatives).
prerequisites_for([K_math_functions,K_math_algebra],L_math_calculus_integrals).
prerequisites_for([K_math_numbers,K_math_functions,K_math_algebra,K_math_geometry],L_math_linear_algebra).
prerequisites_for([K_math_calculus_derivatives,K_math_calculus_integrals],L_math_calculus_multivariable).
prerequisites_for([K_math_calculus_derivatives,K_math_calculus_integrals,K_math_calculus_multivariable,K_math_linear_algebra],L_math_calculus_vector).
prerequisites_for([K_math_calculus_derivatives,K_math_calculus_integrals,K_math_calculus_multivariable],L_math_probability).
prerequisites_for([K_math_probability],L_math_statistics).
prerequisites_for([K_math_calculus,K_math_linear_algebra],L_math_differential_equations).
prerequisites_for([K_math_calculus_derivatives,K_math_calculus_integrals,K_math_linear_algebra,K_math_differential_